In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("ETL_03_Gold").getOrCreate()

# --- Recarregando DataFrames da Silver ---
HDFS_SILVER = "hdfs://namenode:9000/datalake/silver"

df_suicidio_b = spark.read.parquet(f"{HDFS_SILVER}/suicidio_b")
df_suicidio_f = spark.read.parquet(f"{HDFS_SILVER}/suicidio_f")
df_suicidio_m = spark.read.parquet(f"{HDFS_SILVER}/suicidio_m")

df_depressao_b = spark.read.parquet(f"{HDFS_SILVER}/depressao_b")
df_depressao_f = spark.read.parquet(f"{HDFS_SILVER}/depressao_f")
df_depressao_m = spark.read.parquet(f"{HDFS_SILVER}/depressao_m")

# --- Renomeação ---
def renomear_coluna(df, coluna, nome_novo):
    return df.withColumnRenamed(coluna, nome_novo)

print("Renomeando coluna Valor de acordo com a tabela:")

df_suicidio_b = renomear_coluna(df_suicidio_b, "Valor", "Suicidio_Ambos")
df_suicidio_f = renomear_coluna(df_suicidio_f, "Valor", "Suicidio_Mulher")
df_suicidio_m = renomear_coluna(df_suicidio_m, "Valor", "Suicidio_Homem")

df_depressao_b = renomear_coluna(df_depressao_b, "Valor", "Depressao_Ambos")
df_depressao_f = renomear_coluna(df_depressao_f, "Valor", "Depressao_Mulher")
df_depressao_m = renomear_coluna(df_depressao_m, "Valor", "Depressao_Homem")

# --- 5. JOIN (Camada GOLD) ---
df_gold = (df_suicidio_b
    .drop("Sexo")
    .join(df_depressao_b.drop("Sexo"), ["Pais", "Ano"], "outer")
    .join(df_suicidio_f.drop("Sexo"), ["Pais", "Ano"], "outer")
    .join(df_depressao_f.drop("Sexo"), ["Pais", "Ano"], "outer")
    .join(df_suicidio_m.drop("Sexo"), ["Pais", "Ano"], "outer")
    .join(df_depressao_m.drop("Sexo"), ["Pais", "Ano"], "outer")
)

print("Resultado do join GOLD:")
df_gold.show(10, truncate=False)

# --- 7. Salvando em PARQUET ---
CAMINHO_GOLD = "hdfs://namenode:9000/datalake/gold/suicidio_depressao"
df_gold.write.mode("overwrite").parquet(CAMINHO_GOLD)
print("Arquivo GOLD salvo em Parquet com sucesso!")

# --- 8. PostgreSQL ---
url = "jdbc:postgresql://postgres:5432/superset"
tabela = "fato_suicidio_depressao"
props = {
    "user": "superset",
    "password": "superset",
    "driver": "org.postgresql.Driver"
}

try:
    df_gold.write.mode("overwrite").jdbc(url=url, table=tabela, properties=props)
    print("Tabela salva com sucesso no PostgreSQL!")
except Exception as e:
    print(f"Erro no Postgres (verifique se o driver JDBC está carregado): {e}")

spark.stop()

Renomeando coluna Valor de acordo com a tabela:
Resultado do join GOLD:
+----+----+--------------+---------------+---------------+----------------+--------------+---------------+
|Pais|Ano |Suicidio_Ambos|Depressao_Ambos|Suicidio_Mulher|Depressao_Mulher|Suicidio_Homem|Depressao_Homem|
+----+----+--------------+---------------+---------------+----------------+--------------+---------------+
|AFG |2021|3.5954195     |5.317680074    |2.362037216    |3.700842902     |4.806042705   |6.973242345    |
|AGO |2021|8.113778244   |14.58163984    |3.850428722    |6.343334707     |12.47141719   |24.43465005    |
|ALB |2021|2.754698399   |2.128192159    |2.325584687    |1.742901348     |3.193044245   |2.560050618    |
|ARE |2021|1.706592121   |1.695641491    |0.362485353    |0.531894591     |2.455382054   |2.33446244     |
|ARG |2021|7.938106313   |7.663881144    |3.373439477    |3.36341288      |12.575077     |12.17525612    |
|ARM |2021|2.45846852    |1.92675448     |0.982514438    |0.67570137    